In [2]:

import model_utils
import torch
from models.VGG16 import vgg16

ModuleNotFoundError: No module named 'model_utils'

加载预训练 VGG16 模型并测试分割功能

In [ ]:
# 加载 VGG16 模型
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=False)
model = vgg16()
input_shape = (1, 3, 224, 224)
block_configs, pooling_layers = model_utils.auto_segment_model(model, input_shape)
print('分割得到的块配置:')
for k, v in block_configs.items():
    print(f'{k}: {v}')
print('识别出的池化层:')
for i, pool in enumerate(pooling_layers):
    print(f'Pool {i+1}: {type(pool)}')

0.04650378227233887
0.012612104415893555
0.012401342391967773
0.015110254287719727
0.010510683059692383
0.010999202728271484
0.01059412956237793
0.0049991607666015625
0.009999513626098633
0.006000518798828125
0.0030002593994140625
0.0031881332397460938
0.0028574466705322266
分割得到的块配置:
block_1: {'layers': [(3, 64), (64, 64)], 'in_c': 3, 'out_c': 64, 'width': 224}
block_2: {'layers': [(64, 128), (128, 128)], 'in_c': 64, 'out_c': 128, 'width': 112}
block_3: {'layers': [(128, 256), (256, 256), (256, 256)], 'in_c': 128, 'out_c': 256, 'width': 56}
block_4: {'layers': [(256, 512), (512, 512), (512, 512)], 'in_c': 256, 'out_c': 512, 'width': 28}
block_5: {'layers': [(512, 512), (512, 512), (512, 512)], 'in_c': 512, 'out_c': 512, 'width': 14}
识别出的池化层:
Pool 1: <class 'torch.nn.modules.pooling.MaxPool2d'>
Pool 2: <class 'torch.nn.modules.pooling.MaxPool2d'>
Pool 3: <class 'torch.nn.modules.pooling.MaxPool2d'>
Pool 4: <class 'torch.nn.modules.pooling.MaxPool2d'>
Pool 5: <class 'torch.nn.modules.poo

In [ ]:
import torch
import sys

# 确保 model_utils 和 model_segments 可以在 Python 路径中被找到
try:
    from model_utils import auto_segment_model
    from model_segments import load_segment_models_dynamically
except ImportError:
    print("错误: 请确保 'model_utils.py' 和 'model_segments.py' 文件与此测试脚本在同一目录下。")
    sys.exit(1)


In [ ]:

def run_test_for_model(model_name, input_shape, k, r):
    """
    对指定的模型运行一套完整的分割和加载测试。
    """
    print(f"\n{'='*20} 开始测试: {model_name.upper()} {'='*20}")
    
    try:
        print(f"--> 正在调用 load_segment_models_dynamically for '{model_name}'...")
        master_models, worker_models, pooling_layers = load_segment_models_dynamically(
            model_name=model_name,
            k_workers=k,
            r_workers=r,
            input_shape=input_shape
        )
        print("--> 动态加载成功！")

        num_blocks = len(master_models)
        print(f"\n--- 验证结果 ---")
        print(f"识别出的卷积块数量: {num_blocks}")
        print(f"识别出的池化层数量: {len(pooling_layers)}")
        
        assert num_blocks > 0, f"测试失败: 未能为 '{model_name}' 识别出任何卷积块。"
        assert len(master_models) == len(worker_models), "测试失败: Master 和 Worker 的模型数量不匹配。"
        assert abs(len(master_models) - len(pooling_layers)) <= 1, "测试失败: 池化层和卷积块的数量关系不正确。"

        print("\n--- 详细配置检查 ---")
        for i, block_name in enumerate(master_models.keys()):
            print(f"  - 块 {i+1}: '{block_name}'")
            assert block_name in worker_models, f"测试失败: Worker 模型中缺少 '{block_name}'。"
            
            encoder, final_decoder = master_models[block_name]
            worker_decoder = worker_models[block_name]
            
            assert isinstance(encoder, torch.nn.Module), f"测试失败: '{block_name}' 的 Encoder 不是一个 nn.Module。"
            assert isinstance(final_decoder, torch.nn.Module), f"测试失败: '{block_name}' 的 FinalDecoder 不是一个 nn.Module。"
            assert isinstance(worker_decoder, torch.nn.Module), f"测试失败: '{block_name}' 的 WorkerDecoder 不是一个 nn.Module。"
            print(f"    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。")

        print(f"\n{'='*20} 测试成功: {model_name.upper()} {'='*20}")
        return True

    except Exception as e:
        print(f"\n{'!'*20} 测试失败: {model_name.upper()} {'!'*20}")
        print(f"错误信息: {e}")
        import traceback
        traceback.print_exc()
        return False


In [ ]:

if __name__ == '__main__':
    k, r = 4, 2
    
    # --- 测试 VGG16 ---
    vgg_success = run_test_for_model('vgg16', (1, 3, 224, 224), k, r)
    
    print("\n" + "-"*50 + "\n")
    
    # --- 测试 AlexNet ---
    alexnet_success = run_test_for_model('alexnet', (1, 3, 224, 224), k, r)

    print("\n" + "#"*50)
    if vgg_success and alexnet_success:
        print("所有测试均已通过！您的 model_segments.py 文件工作正常。")
    else:
        print("部分或全部测试失败。请检查上面的错误日志。")
    print("#"*50)




==================== 开始测试: VGG16 ====================
--> 正在调用 load_segment_models_dynamically for 'vgg16'...
0.00507807731628418
0.009376764297485352
0.005201578140258789
0.010150432586669922
0.0060007572174072266
0.00908660888671875
0.009994268417358398
0.006155490875244141
0.009999513626098633
0.010247230529785156
0.00299835205078125
0.0034492015838623047
0.0030002593994140625
模型 'vgg16' 被自动分割为 5 个块。
--> 动态加载成功！

--- 验证结果 ---
识别出的卷积块数量: 5
识别出的池化层数量: 5

--- 详细配置检查 ---
  - 块 1: 'block_1'
    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。
  - 块 2: 'block_2'
    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。
  - 块 3: 'block_3'
    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。
  - 块 4: 'block_4'
    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。
  - 块 5: 'block_5'
    - Encoder, FinalDecoder, WorkerDecoder 实例已成功创建。

==================== 测试成功: VGG16 ====================

--------------------------------------------------


==================== 开始测试: ALEXNET ====================
--> 

Traceback (most recent call last):
  File "C:\Users\Zaoer\AppData\Local\Temp\ipykernel_14300\3380877305.py", line 9, in run_test_for_model
    master_models, worker_models, pooling_layers = load_segment_models_dynamically(
  File "d:\OneDrive\Study\CodedInference\model_segments.py", line 82, in load_segment_models_dynamically
    raise ValueError(f"模型 '{model_name}' 尚不支持自动分割。")
ValueError: 模型 'alexnet' 尚不支持自动分割。
